importer les données et les traiter pour que tout soit numérique

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore') 


data = pd.read_csv("Obesite_train.csv", sep=";")

# pour voir les données vite fait
#print(data.head())

X = data.iloc[:, :-1].values  # toutes les colonnes à savoir
y = data.iloc[:, -1].values  # la colonne à prédire


# variables catégorielles 
col_cat = [1, 5, 6, 9, 10, 12, 15, 16]
X_cat = np.copy(X[:, col_cat])  
for col_id in range(len(col_cat)): 
    unique_val, val_idx = np.unique(X_cat[:, col_id], return_inverse=True) 
    X_cat[:, col_id] = val_idx 
imp_cat = SimpleImputer (missing_values=0, strategy='most_frequent') 
X_cat[:, range(2)] = imp_cat.fit_transform(X_cat[:, range(2)]) 


# variables numériques 
col_num = [0, 2, 3, 4, 7, 8, 11, 13, 14]
X_num = np.copy(X[:, col_num]) 
X_num[X_num == '?'] = np.nan 
X_num = X_num.astype(float) 
imp_num = SimpleImputer (missing_values=np.nan, strategy='mean') 
X_num = imp_num.fit_transform(X_num) 

encoder = OneHotEncoder()

X_cat_bin = encoder.fit_transform(X_cat).toarray() 


# on concatène les variables catégorielles et numériques
X = np.hstack((X_cat_bin, X_num)) 

on sépare les données en train et test 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

on cherche le meilleur classifieur

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, make_scorer

# Liste des modèles

# NaiveBayesSimple 
# Un arbre CART 
# Un arbre ID3 
# Un Decision Stump 
# MultilayerPerceptron à deux couches de tailles respectives 20 et 10 par exemple 
# k-plus-proches-voisins avec k=5 par exemple 
# Bagging avec 200 classifieurs par exemple 
# AdaBoost avec 200 classifieurs par exemple 
# Random Forest avec 200 classifieurs par exemple 
# Un XGboost avec 200 classifieurs par exemple

models = {
    'NB': GaussianNB(),
    'CART': DecisionTreeClassifier(random_state=1),
    'DS': DecisionTreeClassifier(max_depth=1, random_state=1),
    'MLP': MLPClassifier(hidden_layer_sizes=(40, 40, 20, 10), random_state=1),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'BAG':BaggingClassifier(n_estimators=200,  random_state=1),
    'RF': RandomForestClassifier(n_estimators=200, random_state=1),
    'AB': AdaBoostClassifier(n_estimators=200, random_state=1),
    'XGB': GradientBoostingClassifier(n_estimators=200, random_state=1)
}

def custom_cost(y_true, y_pred):
    recall_0 = recall_score(y_true, y_pred, pos_label=0) 
    recall_1 = recall_score(y_true, y_pred, pos_label=1)
    return 0.5 * recall_0 + 0.5 * recall_1

# création du scorer (pour personnaliser ton truc à maximiser)
# dedans je mets la fonction du score donnée dans l'énoncé
scorer = make_scorer(custom_cost, greater_is_better=True)

# fonction d'évaluation des modèles    
    
def run_classifiers(clfs, X, Y):
    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    for i in clfs:
        clf = clfs[i]
        cv_acc = cross_val_score(clf, X, Y, cv=kf, scoring=scorer)
        print("Accuracy for {0} is: {1:.3f} +/- {2:.3f}".format(i, np.mean(cv_acc), np.std(cv_acc)))
        

print("on compare les modèles (data d'origine)")
run_classifiers(models, X, y)

#data normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

print("on compare les modèles (data normalisé)")
run_classifiers(models, X_train_scaled, y_train)

on trouve que le meilleur classifieur est AB donc on cherche ses meilleurs paramètres

In [ ]:
from sklearn.model_selection import GridSearchCV

# Définir les paramètres à optimiser pour AB
parametres_a_optimiser = {
    "n_estimators": [50, 100, 200, 400],
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.5],
    "random_state": [1]  # ne bouge pas
}

# Instancier GridSearchCV pour trouver les meilleurs paramètres
grid_search = GridSearchCV(
    AdaBoostClassifier(),
    parametres_a_optimiser,
    scoring=scorer,
    cv=3,  # Validation croisée à 3 plis
    n_jobs=-1,  # Utiliser tous les cœurs disponibles
    verbose=1  # Affiche les détails de l'optimisation
)


# Ajuster le modèle sur les données d'entraînement
grid_search.fit(X_train, y_train)

print("Meilleurs paramètres :", grid_search.best_params_)

on obtient ça : "Meilleurs paramètres : {'learning_rate': 0.1, 'n_estimators': 100, 'random_state': 1}"

In [ ]:
# du coup full training sur le meilleur modele 
final_model = AdaBoostClassifier(learning_rate =0.1, n_estimators=100, random_state=1)
final_model.fit(X, y)

et maintenant on peut faire les prédictions

In [ ]:

final = pd.read_csv("Obesite_scoring.csv", sep=";")

# on applique les mêmes transformations que pour les données d'entraînement

# variables catégorielles
X_scoring_cat = np.copy(final.iloc[:, col_cat].values)
for col_id in range(len(col_cat)):
    unique_val, val_idx = np.unique(X_scoring_cat[:, col_id], return_inverse=True)
    X_scoring_cat[:, col_id] = val_idx
X_scoring_cat[:, range(2)] = imp_cat.transform(X_scoring_cat[:, range(2)])


# variables numériques
X_scoring_num = np.copy(final.iloc[:, col_num].values)
X_scoring_num[X_scoring_num == '?'] = np.nan
X_scoring_num = X_scoring_num.astype(float)
X_scoring_num = imp_num.transform(X_scoring_num)

X_scoring_cat_bin = encoder.transform(X_scoring_cat).toarray()
X_scoring_prepared = np.hstack((X_scoring_cat_bin, X_scoring_num))

# Prédiction sur le fichier de scoring
y_scoring_pred = final_model.predict(X_scoring_prepared)

# Création du fichier de sortie avec Id et classe
scoring_results = pd.DataFrame({
    "Id": final["Id"],  
    "Obesite?": y_scoring_pred
})

# on sauvegarde dans un fichier csv
scoring_results.to_csv("Obesite_scoring_predictions.csv", index=False, sep=";", columns=["Id", "Obesite?"])